<a href="https://colab.research.google.com/github/cezac/computacao_Distribuida/blob/main/C%C3%B3pia_de_runningMPI_in_Colab.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [17]:
!mpicc -o teste-mpi teste-mpi.c

In [18]:
! mpirun -n 4 -H  localhost,localhost,localhost,localhost --allow-run-as-root ./teste-mpi

Valor 135862 encontrado na posição 10.


In [16]:
%%writefile teste-mpi.c
#include "mpi.h"
#include <stdio.h>
#include <math.h>
#include <unistd.h>
#include <stdlib.h>

#define MAXSIZE 65536 // Valor de 2^16

int main(int argc, char **argv) {
    int myid, numprocs;
    int data[MAXSIZE], i, x, low, high;
		int target = 135862; // Valor buscado no vetor
    int found = 0; // Variável para indicar se o valor foi encontrado
    int found_index = -1; // Variável para armazenar a posição do valor encontrado
    FILE *fp;
    MPI_Init(&argc, &argv);
    MPI_Comm_size(MPI_COMM_WORLD, &numprocs);
    MPI_Comm_rank(MPI_COMM_WORLD, &myid);

    if (myid == 0) {  /* Open input file and initialize data */
        if ((fp = fopen("vetor1.csv", "r")) == NULL) {
            printf("Não foi possível abrir o arquivo: vetor1.csv\n\n");
            exit(1);
        }
        for(i = 0; i < MAXSIZE; i++) fscanf(fp, "%d\n", &data[i]);
    }

    MPI_Bcast(data, MAXSIZE, MPI_INT, 0, MPI_COMM_WORLD); /* broadcast data */

    x = MAXSIZE / numprocs; /* Add my portion of data */
    low = myid * x;
    high = low + x;

    for (i = low; i < high; i++) {
        if (data[i] == target) {
            found = 1; // Indica que o valor foi encontrado
            found_index = i; // Armazena a posição do valor encontrado
            break; // Não é necessário continuar procurando
        }
    }

    // Envia o resultado para o processo "Master"
    if (myid != 0) {
        MPI_Send(&found, 1, MPI_INT, 0, 0, MPI_COMM_WORLD);
        MPI_Send(&found_index, 1, MPI_INT, 0, 0, MPI_COMM_WORLD);
    } else {
        for (int proc = 1; proc < numprocs; proc++) {
            int temp_found, temp_index;
            MPI_Recv(&temp_found, 1, MPI_INT, proc, 0, MPI_COMM_WORLD, MPI_STATUS_IGNORE);
            MPI_Recv(&temp_index, 1, MPI_INT, proc, 0, MPI_COMM_WORLD, MPI_STATUS_IGNORE);
            if (temp_found) {
                found = 1;
                found_index = temp_index;
                break;
            }
        }

        if (found) {
            printf("Valor %d encontrado na posição %d.\n", target, found_index);
        } else {
            printf("Valor %d não encontrado.\n", target);
        }
    }

    MPI_Finalize();
    return 0;
}

Overwriting teste-mpi.c


In [9]:
!apt install openmpi-bin openmpi-common libopenmpi-dev

Reading package lists... Done
Building dependency tree... Done
Reading state information... Done
libopenmpi-dev is already the newest version (4.1.2-2ubuntu1).
openmpi-bin is already the newest version (4.1.2-2ubuntu1).
openmpi-common is already the newest version (4.1.2-2ubuntu1).
0 upgraded, 0 newly installed, 0 to remove and 16 not upgraded.
